In [1]:
import os
import queue
from ipywidgets import Textarea, Play, GridBox, GridspecLayout, Box
from fullscreen import Fullscreen
from IPython.display import display, HTML, clear_output, Javascript
import nglview as nv
import mdtraj as md
import time
import multiprocessing
from idle_time import IdleMonitor
import threading
import pyautogui
from pynput import keyboard

global user_input

user_input=None

def on_press(key):
    global user_input
    if key == keyboard.Key.esc:
        user_input = 'esc'
        
 
def on_release(key):
    if key == keyboard.Key.esc:
        # Stop listener
        return False

def FakeKeyboardInput():
    pyautogui.keyDown('shift')
    time.sleep(.2)
    pyautogui.keyDown('enter')
    time.sleep(.2)
    pyautogui.keyUp('shift')
    time.sleep(.2)
    pyautogui.keyUp('enter')

def UpdateViewer(view,traj):
    view.clear()
    view.add_trajectory(traj)
    
def ConvertSecondsToMinutes(currenttime):
    return currenttime/60
    
def StartPlayer(traj,view,trajectory_path,delaytime,screensleeptime,fs,box):
    monitor = IdleMonitor.get_monitor()
    p=None
    while True:
        idletime=ConvertSecondsToMinutes(monitor.get_idle_time())
        if user_input=='esc':
            fs._is_fullscreen=False
        if idletime>=screensleeptime and fs._is_fullscreen==False:
            break

        if p==None:
            if os.path.exists('readytoappend.txt'):
                os.remove('readytoappend.txt')
                p = multiprocessing.Process(target=LoadTrajectory, args=(trajectory_path,))
                p.start()
            else:
                frames=traj.n_frames 
                PlayTrajectory(frames,delaytime,view)
        else:
            frames=traj.n_frames 
            while p.is_alive():
                idletime=ConvertSecondsToMinutes(monitor.get_idle_time())
                if user_input=='esc':
                    fs._is_fullscreen=False
                if idletime>=screensleeptime and fs._is_fullscreen==False:
                    break
                PlayTrajectory(frames,delaytime,view)
            if idletime>=screensleeptime:
                break
            p.join()
            UpdateViewer(view,traj)
            frames=traj.n_frames  
            p=None
    return fs,traj,view
        
    
                
def PlayTrajectory(frames,delaytime,view):
    for i in range(1,frames+1):
        time.sleep(delaytime)
        view.frame=i
        
def LoadTrajectory(trajectory_path):
    traj=md.load_arc(trajectory_path)
    return traj

def DisplayBox(box):
    display(box)
    
    
def DisplayFullView(v):
    v.layout.width = '100%'
    b = GridBox([Box([v])])
    fs = Fullscreen(b, [v])
    display(fs)

    b.add_class('abc')
    #v.handle_resize()
    display(HTML("""
    <style>
    .abc {
        background: white
    }
    </style>
    """))
    thread = threading.Thread(target=DisplayBox, args=(b,))
    thread.start()
    
    return b,v,fs



delaytime=.05
screensleeptime=.1
BASE_DIR = os.path.abspath('')
xyzfilename='initialpdb.xyz'
xyzfile_path=BASE_DIR+'/'+xyzfilename
trajectory_path=xyzfile_path.replace('.xyz','.arc')

traj=LoadTrajectory(trajectory_path)
view = nv.show_mdtraj(traj)
box,view,fs=DisplayFullView(view)
    
listener = keyboard.Listener(on_press=on_press,on_release=on_release)
listener.start()
display(Javascript("Jupyter.notebook.execute_cells([1])"))







Fullscreen()

GridBox(children=(Box(children=(NGLWidget(layout=Layout(width='100%'), max_frame=9),)),), _dom_classes=('abc',…

<IPython.core.display.Javascript object>

In [3]:
fs.fullscreen()
FakeKeyboardInput()

In [7]:
fs,traj,view=StartPlayer(traj,view,trajectory_path,delaytime,screensleeptime,fs,box)
listener = keyboard.Listener(on_press=on_press,on_release=on_release)
listener.start()
FakeKeyboardInput()
fs.fullscreen()


here
user_input esc


KeyboardInterrupt: 

In [5]:
display(Javascript("Jupyter.notebook.execute_cells([2])"))

<IPython.core.display.Javascript object>